In [3]:
import tensorflow as tf;
from tensorflow.examples.tutorials.mnist import input_data;

In [ ]:
#input data set
mnist = input_data.read_data_sets("MNIST_data",one_hot=True);

#input images row and col is 28*28
n_inputs=28;#one row have 28 data
max_time=28;#one col have 28 data
lstm_size=100;#hidden 隐层单元
n_classes=10;#10 calsses 10个分类
batch_size=50;#every batch have 50 features
n_batch=mnist.train.num_examples;#calculate have how much bacth

x=tf.placeholder(tf.float32,[None,784]);
y=tf.placeholder(tf.float32,[None,10]);

#initialise variable
weights = tf.Variable(tf.truncated_normal([lstm_size,n_classes],stddev=0.1));
biases = tf.Variable(tf.constant(0.1,shape=[n_classes]));

#define RNN network
def RNN(X,weights,biases):
    #inputs=[batch_size,max_time,n_inputs]
    inputs=tf.reshape(X,[-1,max_time,n_inputs]);
    #define LSTM basic type cell
    #lstm_cell = tf.contrib.rnn.core_rnn_cell.BasicLSTMCell(lstm_size);
    lstm_cell = tf.contrib.rnn.BasicLSTMCell(lstm_size);
    outputs,final_state=tf.nn.dynamic_rnn(lstm_cell,inputs,dtype=tf.float32);
    results= tf.nn.softmax(tf.matmul(final_state[1],weights+biases));
    return results;
#calculate RNN result
prediction = RNN(x,weights,biases);
#损失函数
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction,labels=y));
#使用AdamOptimizer进行优化
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy);
#结果存放在一个布尔型列表中
correct_prediction=tf.equal(tf.argmax(y,1),tf.argmax(prediction,1));
#求准确率# 把correct_prediction变为float32类型
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32));
init = tf.global_variables_initializer();

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(6):
        for batch in range(n_batch):
            batch_xs,batch_ys =  mnist.train.next_batch(batch_size)
            sess.run(train_step,feed_dict={x:batch_xs,y:batch_ys})
        
        acc = sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels})
        print ("Iter " + str(epoch) + ", Testing Accuracy= " + str(acc))


Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz
